In [1]:
import matplotlib
import numpy as np
import PIL
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import coremltools
from pycoreimage.pyci import cimg, show, color, inset

In [2]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [30,30]
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% : important; }</style>"))
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [8]:
# print(cimg.inputs('CICMYKHalftone'))
# for i,f in enumerate(cimg.filters()):
#     print('{:3d}\t {}'.format(i,f))

In [9]:
model = coremltools.models.MLModel('models/DeepLab.mlmodel')
labels = ['Background','Plane','Bicycle','Bird','Boat','Bottle','Bus','Car','Cat','Chair','Cow','Diningtable','Dog','Horse','Motorbike','Person','Pottedplant','Sheep','Sofa','Train','Tvmonitor']
dictionary = {'Background':0,'Plane':1,'Bicycle':2,'Bird':3,'Boat':4,'Bottle':5,'Bus':6,'Car':7,'Cat':8,'Chair':9,'Cow':10,'Diningtable':11,'Dog':12,'Horse':13,'Motorbike':14,'Person':15,'Pottedplant':16,'Sheep':17,'Sofa':18,'Train':19,'Tvmonitor':20}
spec = model._spec
input_width = spec.description.input[0].type.imageType.width
input_height = spec.description.input[0].type.imageType.height

# TODO: import random image from unsplash
img = PIL.Image.open('samples/people_woman_2.jpg')
image_width = np.array(img).shape[1]
image_height = np.array(img).shape[0]
resized_img = img.resize((input_width, input_height), PIL.Image.BILINEAR)
y = model.predict({"image": resized_img}, usesCPUOnly=False)
mask = y['scores']

scale_width = image_width/input_width
scale_height = image_height/input_height
ci_mask_resized = cimg(np.array(mask)).scale(scale_width,scale_height)
ci_fg = cimg(np.array(img))
ci_bg = cimg(np.array(img))

src = """ kernel vec4 color(__sample img, __sample mask, float index) {
    vec4 pixValue;
    pixValue.r = pixValue.g = pixValue.b = pixValue.a = 0;
    if (mask.r >= index - 0.0039 && mask.r <= index + 0.0039){
        pixValue.r = pixValue.g = pixValue.b = pixValue.a = 1;
    }
    return pixValue;
}"""

bilateral_filter = """ kernel vec4 bilateral(sampler u, sampler fg, float k, float colorInv, float spatialInv)
{
  vec2 dc = destCoord();
  vec2 pu = samplerCoord(u);
  vec2 uDelta = samplerTransform(u, dc+vec2(1.0)) - pu;
  vec4 u_0 = sample(u, pu);

  vec2 pfg = samplerCoord(fg);
  vec2 fgDelta = samplerTransform(fg, dc+vec2(1.0)) - pfg;
  vec4 fg_0 = sample(fg, pfg);

  vec4 C = vec4(0.0);
  float W = 0.0;
  for (float x = -k; x <= k; x++) {
    for (float y = -k; y <= k; y++){
      float ws = exp(-(x*x+y*y) * spatialInv);
      vec4 u_xy  = sample(u, pu + vec2(x,y)*uDelta);
      vec4 fg_xy  = sample(fg, pfg + vec2(x,y)*fgDelta);
      vec3 diff = fg_xy.rgb-fg_0.rgb;
      float wc = exp(-dot(diff,diff) * colorInv);
      W += ws * wc;
      C += ws * wc * u_xy;
    }
  }
  return W < 0.0001 ? u_0 : C / W;  
}
"""

In [10]:
indexValue = dictionary['Person']*12/255.0
ci_mask_blured = ci_mask_resized.applyKernel(bilateral_filter, ci_fg, 10, 2000,0.001)
# show([result])

## Highlight Template

## Color Highlight

In [6]:
# print(cimg.inputs('gaussianBlur'))
def color(img, mask, contrast, saturation):
    ci_fg = img.colorControls(contrast=1.2, saturation = 1.2)
    ci_bg = img.colorControls(contrast=contrast, saturation = saturation)
    output = ci_fg.blendWithMask(backgroundImage=ci_bg, maskImage=ci_mask_blured)
    return output
    
def show_color(image, mask, contrast, saturation):
    output = color(image, mask, contrast, saturation)
    show(output)
    
interact(show_color, image = fixed(ci_bg), 
         mask = fixed(ci_mask_blured), 
         contrast=widgets.FloatSlider(min=0,max=2,step=0.05,value=0.7),
        saturation=widgets.FloatSlider(min=0,max=2,step=0.05,value=0.7));

interactive(children=(FloatSlider(value=0.7, description='contrast', max=2.0, step=0.05), FloatSlider(value=0.…

## Bokeh Blur Hilight

In [7]:
# print(cimg.inputs('gaussianBlur'))
def bokeh(img, mask, radius):
    ci_fg = img
    ci_bg = img.gaussianBlur(radius=radius)
    output = ci_fg.blendWithMask(backgroundImage=ci_bg, maskImage=ci_mask_blured)
    return output
    
def show_bokeh(image, mask, radius):
    output = bokeh(image, mask, radius)
    show(output)
    
interact(show_bokeh, image = fixed(ci_bg), 
         mask = fixed(ci_mask_blured), 
         radius=widgets.IntSlider(min=1,max=30,step=1,value=10));

interactive(children=(IntSlider(value=10, description='radius', max=30, min=1), Output()), _dom_classes=('widg…